In [4]:
import csv
import random
import numpy as np

In [5]:
with open('/content/heart.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

# Headers - Column Labels
header = data[0]
header[0] = "age"

# Dataset
data = data[1:]

# Randomize and then Split
random.shuffle(data)
size = len(data)

training_data = data[:int(size*0.7)]  
testing_data = data[int(size*0.7):]  


In [6]:
print(training_data)

[['58', '1', '2', '112', '230', '0', '0', '165', '0', '2.5', '1', '1', '3', '0'], ['35', '0', '0', '138', '183', '0', '1', '182', '0', '1.4', '2', '0', '2', '1'], ['60', '1', '0', '130', '206', '0', '0', '132', '1', '2.4', '1', '2', '3', '0'], ['60', '0', '2', '120', '178', '1', '1', '96', '0', '0', '2', '0', '2', '1'], ['57', '1', '0', '110', '201', '0', '1', '126', '1', '1.5', '1', '0', '1', '1'], ['58', '0', '3', '150', '283', '1', '0', '162', '0', '1', '2', '0', '2', '1'], ['59', '1', '3', '160', '273', '0', '0', '125', '0', '0', '2', '0', '2', '0'], ['61', '1', '3', '134', '234', '0', '1', '145', '0', '2.6', '1', '2', '2', '0'], ['60', '1', '0', '117', '230', '1', '1', '160', '1', '1.4', '2', '2', '3', '0'], ['54', '1', '0', '124', '266', '0', '0', '109', '1', '2.2', '1', '1', '3', '0'], ['57', '1', '2', '150', '168', '0', '1', '174', '0', '1.6', '2', '0', '2', '1'], ['48', '1', '0', '130', '256', '1', '0', '150', '1', '0', '2', '2', '3', '0'], ['58', '1', '2', '132', '224', '0', 

In [7]:
class HelperFunctions:
    def __init__(self):
        pass
    
    """Find the unique values for a column in a dataset."""
    def unique_vals(self, rows, col):
        return set([row[col] for row in rows])
    
    """Count No of each Unique Val for the prediction."""
    def class_counts(self, rows):
        counts = {}
        for row in rows:
            label = row[-1]
            if label not in counts:
                counts[label] = 0
            counts[label] += 1
        return counts
    
    """Check whether a value is numeric or not"""
    def is_numeric(self, value):
        return isinstance(value, int) or isinstance(value, float)
    
    def partition(self, rows, question):
        """
        Partitions a dataset. For each row in the dataset, 
        check if it matches the question. If so, add it to 
        'true rows', otherwise, add it to 'false rows'.
        """
        true_rows, false_rows = [], []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows


In [8]:

helper = HelperFunctions()

In [9]:
class Question:
    """A Question is used to partition a dataset."""

    def __init__(self, column, value, unique_count = None):
        self.column = column
        self.value = value
        self.unique_count = helper.unique_vals(training_data, column)

    def match(self, example):
        val = example[self.column]
        if len(self.unique_count)<=10:
            return val == self.value
        else:
            return val >= self.value

    def __repr__(self):
        # Printing the question in a readable format.
        condition = "=="
        if len(self.unique_count)>10:
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [10]:
def gini(rows):
    counts = helper.class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


In [11]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = helper.partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [12]:
best_gain, best_question = find_best_split(training_data)
best_question

Is cp == 0?

In [13]:
class Leaf:
    def __init__(self, rows):
        self.predictions = helper.class_counts(rows)


In [14]:

class Decision_Node:

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
def build_tree(rows):
  
    gain, question = find_best_split(rows)

   
    if gain == 0:
        return Leaf(rows)

   
    true_rows, false_rows = helper.partition(rows, question)

  
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    
    return Decision_Node(question, true_branch, false_branch)

In [16]:
def print_tree(node, spacing=""):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "    ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "    ")

In [17]:
my_tree = build_tree(training_data)


In [18]:
print_tree(my_tree)

Is cp == 0?
--> True:
    Is ca == 0?
    --> True:
        Is thal == 2?
        --> True:
            Is age >= 59?
            --> True:
                Is chol >= 271?
                --> True:
                    Is chol >= 330?
                    --> True:
                        Is age >= 62?
                        --> True:
                            Predict {'1': 2}
                        --> False:
                            Predict {'0': 1}
                    --> False:
                        Predict {'1': 8}
                --> False:
                    Is age >= 71?
                    --> True:
                        Predict {'1': 3}
                    --> False:
                        Is slope == 2?
                        --> True:
                            Predict {'1': 2}
                        --> False:
                            Predict {'0': 12}
            --> False:
                Is trestbps >= 180?
                --> True:
                    

In [24]:
def classify(row, node):
  
    if isinstance(node, Leaf):
        return node.predictions

    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [25]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


In [26]:
y_true = []
y_pred = []
for row in testing_data:
    actual = row[-1]
    pred = print_leaf(classify(row, my_tree))
    y_true.append(actual)
    y_pred.append(list(pred.keys())[0])
    print ("Actual: %s. Predicted: %s" %
           (actual, pred))


Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 0. Predicted: {'0': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': '100%'}
Actual: 1. Predicted: {'1': 

In [27]:
def accuracy(y_true,y_pred,normalize=True):
    accuracy=[]
    for i in range(len(y_pred)):
        if y_pred[i]==y_true[i]:
            accuracy.append(1)
        else:
            accuracy.append(0)
    if normalize==True:
        return np.mean(accuracy)
    if normalize==False:
        return sum(accuracy)

In [28]:
accuracy(y_true,y_pred)

1.0